In [1]:
!pip install torchvision
!pip install transformers>=4.25.1
!pip install datasets

from datasets import load_dataset, Image
import torch

from google.colab import drive
drive.mount('/content/drive')

import os
import json
from PIL import Image


#sys.path.append('/opt/cocoapi/PythonAPI')

# https://github.com/L1aoXingyu/image-caption-project/blob/master/0_Dataset.ipynb
# returns a COCO object(which is dataset to be sampled from), coco_caps(captions), and the ids for COCO data instances
def load_coco():
    # initialize COCO API for instance annotations
    dataDir = '/content/gdrive/My Drive/cocoapi'
    dataType = 'val2014'
    instances_annFile = os.path.join(dataDir, 'annotations/instances_{}.json'.format(dataType))
    coco = COCO(instances_annFile)

    # initialize COCO API for caption annotations
    captions_annFile = os.path.join(dataDir, 'annotations/captions_{}.json'.format(dataType))
    coco_caps = COCO(captions_annFile)

    # get image ids
    ids = list(coco.anns.keys())
    return coco, coco_caps, ids


def sample_image(coco, coco_caps, ids, count):
    images = []
    captions = []
    while count > 0:
      count -= 1
      # pick a random image and obtain the corresponding URL
      ann_id = np.random.choice(ids)
      print(f"ann_id: {ann_id}")
      img_id = coco.anns[ann_id]['image_id']
      img = coco.loadImgs(img_id)[0]
      url = img['coco_url']

    # print URL and visualize corresponding image
    #print(url)
    #I = io.imread(url)
    #plt.axis('off')
    #plt.imshow(I)
    #plt.show()

    # load and display captions
      annIds = coco_caps.getAnnIds(imgIds=img['id']);
      anns = coco_caps.loadAnns(annIds)
      #coco_caps.showAnns(anns)
      images.append(img)
      captions.append(anns)
    return images, captions

# generate instructions for LLM, feed into LLM, get optimized prompts
# few shot prompting: type/color of object, posture of object, place of object, placement relationship between objects, "invent" related unmentioned things!
def improve_captions(captions):
    improved_captions  = []
    for cap in captions:
        improved = "This is " + cap # TODO: d
        improved_captions.append(improved)
    return improved_captions

def load_from_local():
    image_folder = '/content/drive/My Drive/C5470prj/coco_subset'
    file_path = image_folder + '/captions_with_improved.json'
    with open(file_path, 'r') as file:
        data = json.load(file)
    ids = []
    images = []
    captions = []
    improved_captions = []
    data = sorted(data, key=lambda x: x['image_id'])
    for item in data:
        ids.append(item['image_id'])
        captions.append(item['caption'])
        improved_captions.append(item['improved_caption'])
        image_id = item['image_id']
        filename = f'{image_id:012d}.jpg' # 12 digit padding
        full_path = os.path.join(image_folder, filename)
        try:
            img = Image.open(full_path)
            images.append(img)
        except FileNotFoundError:
            print(f"Image file not found: {full_path}")
    return ids, images, captions, improved_captions

#coco, coco_caps, ids = load_coco()
#images, captions = sample_images(coco, coco_caps, ids, 1)
#improved_catpions = improve_captions(captions)

!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
import clip
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
from collections import OrderedDict

import numpy as np



  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

100%|███████████████████████████████████████| 338M/338M [00:17<00:00, 20.0MiB/s]


NameError: name 'device' is not defined

In [2]:
device = "cuda"
clip_model, preprocess = clip.load("ViT-B/32")
clip_model.to(device)
clip_model.eval()
input_resolution = clip_model.visual.input_resolution
context_length = clip_model.context_length
vocab_size = clip_model.vocab_size

print(f"Model parameters: {np.sum([int(np.prod(p.shape)) for p in clip_model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

Model parameters: 151,277,313
Input resolution: 224
Context length: 77
Vocab size: 49408


In [8]:
from IPython.display import display
import fnmatch

ids, images, captions, improved_captions = load_from_local()

improved_images = []
improved_folder = '/content/drive/My Drive/C5470prj/improved_images'


def calculate_clip_scores(model, preprocess, images, captions):
    image_features = []
    for image in images:
        image_processed = preprocess(image).unsqueeze(0).to(device)
        image_features.append(model.encode_image(image_processed))
        #display(image)

    print(captions)

    text_tokens = clip.tokenize(captions).to(device)
    text_features = model.encode_text(text_tokens)

    image_features = torch.cat(image_features)
    similarity = torch.matmul(image_features, text_features.t()).diag().cpu().detach().numpy()
    return similarity

def print_stats(improved_images, captions, improved_scores):
    # show prompt-image pairs with highest/lowest score
    paired_scores = list(zip(improved_images, captions, improved_scores))
    paired_scores.sort(key=lambda x: x[2], reverse=True)

    img, cap, sc = paired_scores[0]
    print(f"Highest CLIP Score: {sc}, caption: {cap}, img: ")
    display(img)
    img, cap, sc = paired_scores[-1]
    print(f"Lowest CLIP Score: {sc}, caption: {cap}, img: ")
    display(img)

def load_and_sort(folder_path):
    images = []
    files = sorted(os.listdir(folder_path))
    for id in ids:
        patterns = [f'*{id}*.jpg', f'*{id}*.png']
        for file in files:
            if any(fnmatch.fnmatch(file, pattern) for pattern in patterns):
                images.append(Image.open(os.path.join(folder_path, file)))
                break
    return images

folder_paths = {
    'improved_caption': '/content/drive/My Drive/C5470prj/improved_caption',
    'improved_images': '/content/drive/My Drive/C5470prj/improved_images',
    'original_caption': '/content/drive/My Drive/C5470prj/original_caption'
}

improved_caption_images = load_and_sort(folder_paths['improved_caption'])
improved_images = load_and_sort(folder_paths['improved_images'])
original_caption_images = load_and_sort(folder_paths['original_caption'])


# calculate CLIP-SCORE, again
improved_scores = calculate_clip_scores(clip_model, preprocess, improved_caption_images, captions)
original_scores = calculate_clip_scores(clip_model, preprocess, original_caption_images, captions)

print(f"original avg score: {np.average(original_scores)}")

print(f"imrpoved avg score: {np.average(improved_scores)}")


['Two apple computers are on a white desk', 'a red and black train is coming down the tracks, with workers', 'Goat standing on top of rock pile with grass growing on it', 'A pizza served on a white plate next to empty wine glass and beer.', 'A laptop on a table outside a balcony overlooking the ocean.', 'A bathroom with a sink, toilet and tub in front of a large mirror.', 'A large cat sitting in a bathroom sink under a mirror, staring.', 'A cutting board and knife, with chopped carrots, celery, and onion.', 'some elephants and one is by some water', 'a metal bench resting on a sidewalk with cars on a street in the background', 'A stack of pancakes that are sitting on a plate.', 'A boat passes by waterfront houses flanked by trees.', 'A train passing through wooded areas on a train track.', 'The inside view of a large decorated church.', 'A bowl of Asian Cuisine with beef, noodles and broccoli.']
['Two apple computers are on a white desk', 'a red and black train is coming down the track